In [1]:
import pandas as pd
from IPython.display import clear_output

In [2]:
import time
import random

In [3]:
df = pd.read_csv('actual_name_info_detailed.csv')
df.head()

,Name,School,Power,Link,Hometown
0,Tiago ASAKAWA,UMBC,15.96,/swimmer/515075,"Sao Paulo, BRA"
1,Niklas Weigelt,UMBC,8.38,/swimmer/413265,"Clayton, CA"
2,Ben Beldner,Binghamton,22.71,/swimmer/394588,"Pittsford, NY"
3,Justin Meyn,Binghamton,10.06,/swimmer/471360,"Brightwaters, NY"
4,Joshua Franco,NJIT,12.40,/swimmer/395308,"hillsborough, NJ"


In [4]:
import requests
import csv
from bs4 import BeautifulSoup as bs
import pandas as pd
import time as _time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import re

def getpi(swimmer_name):
    swimmer_name_url = 'https://swimcloud.com/recruiting/rankings/?name='+swimmer_name.replace(' ','+')
    name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
    name_url.encoding = 'utf-8'
    name_soup = bs(name_url.text, 'html.parser')
    # print(name_soup)
    # print(name_soup)
    # print(name_url.text)
    names=[]
    links=[]
    idxs=[]
    teams=[]
    hometowns=[]
    if (name_soup.find('tbody') == None):
        # print('No tbody')
        return None
    swimmer_list = name_soup.find('tbody').find_all('tr')
    i=0
    for swimmer in swimmer_list:
        if len(name_soup.find_all('td', {'class' : 'u-text-end'})) == 0:
            # print('No idx found')
            break
        name = swimmer.find(class_='u-text-semi')
        name_str = str(name)
        start = name_str.find('>') + 1
        end = name_str.find('</a>')
        # Extract the text between '>' and '<'
        name_raw = name_str[start:end]
        if (name_raw.lower() == swimmer_name.lower()):
            names.append(name_raw)
            swimmer_link = name['href'] if name else ''
            links.append(swimmer_link)
            if (swimmer_name == 'Reese Samuel'):
                print(i)
                print(name_soup.find_all('td', {'class' : 'u-text-end'})[i])
            idxs.append(float(name_soup.find_all('td', {'class' : 'u-text-end'})[i].text.strip()))
            team_td = swimmer.find_all('td', class_='u-text-center hidden-xs')[0]
            # Find the <img> tag within the <td>
            img_tag = team_td.find('img')
            if img_tag == None:
                teams.append('')
            else:
                # Extract the 'alt' attribute value, which contains the team name
                team_name = img_tag['alt'][:-5]
                teams.append(team_name)
    
    
            swimmer_name_url = 'https://swimcloud.com'+swimmer_link
            name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
            name_url.encoding = 'utf-8'
            name_soup2 = bs(name_url.text, 'html.parser')
            if (name_soup2.find('ul', class_='o-list-inline o-list-inline--dotted') == None):
                return None
            ul = name_soup2.find('ul', class_='o-list-inline o-list-inline--dotted')
            location_li = ul.find_all('li')[0]
            
            # Extract and print the location text
            location = location_li.text.strip()
            hometowns.append(location)
        i = i + 1
    # print(names)
    # print(links)
    # print(hometowns)
    return [names,links,idxs,teams,hometowns]
